# Further exploration of official FPL API

# Set-up

In [1]:
import os
import requests
import getpass
import pandas as pd
import numpy as np
import json

In [2]:
os.chdir('../..')

In [3]:
from src.data.live_season_data import _get_fpl_json

In [4]:
TEAM_ID = 2852128

In [5]:
MY_TEAM_URL = 'https://fantasy.premierleague.com/api/my-team/{}/'

# Access API endpoints which require authentication

Existing method will throw an HTTPError

In [6]:
_get_fpl_json(MY_TEAM_URL.format(TEAM_ID))

HTTPError: HTTP Error 403: Forbidden

Some parts of API require authentication. This can be achieved by following the steps outlined in this guide: https://medium.com/@bram.vanherle1/fantasy-premier-league-api-authentication-guide-2f7aeb2382e4

In [7]:
session = requests.session()

In [8]:
response = session.get(MY_TEAM_URL.format(TEAM_ID))
print(response)

<Response [403]>


In [9]:
url = 'https://users.premierleague.com/accounts/login/'

fpl_email = getpass.getpass(prompt='Email: ')
fpl_password = getpass.getpass(prompt='Password: ')

payload = {
 'password': fpl_password,
 'login': fpl_email,
 'redirect_uri': 'https://fantasy.premierleague.com/a/login',
 'app': 'plfpl-web'
}

session.post(url, data=payload)

Email:  ···························
Password:  ·········


<Response [200]>

In [10]:
response = session.get(MY_TEAM_URL.format(TEAM_ID))
print(response)

<Response [200]>


In [11]:
response.status_code == 200

True

In [12]:
team_data_json = json.loads(response.text)
team_data_json

{'picks': [{'element': 93,
   'position': 1,
   'selling_price': 47,
   'multiplier': 1,
   'purchase_price': 46,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 122,
   'position': 2,
   'selling_price': 53,
   'multiplier': 1,
   'purchase_price': 53,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 182,
   'position': 3,
   'selling_price': 75,
   'multiplier': 1,
   'purchase_price': 72,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 183,
   'position': 4,
   'selling_price': 65,
   'multiplier': 1,
   'purchase_price': 65,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 443,
   'position': 5,
   'selling_price': 54,
   'multiplier': 1,
   'purchase_price': 54,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 215,
   'position': 6,
   'selling_price': 103,
   'multiplier': 2,
   'purchase_price': 101,
   'is_captain': True,
   'is_vice_captain': False},
  {'element': 29,
   'position'

In [13]:
team_data_json.keys()

dict_keys(['picks', 'chips', 'transfers'])

In [14]:
pd.json_normalize(team_data_json, 'picks')

,element,position,selling_price,multiplier,purchase_price,is_captain,is_vice_captain
0,93,1,47,1,46,False,False
1,122,2,53,1,53,False,False
2,182,3,75,1,72,False,False
3,183,4,65,1,65,False,False
4,443,5,54,1,54,False,False
5,215,6,103,2,101,True,False
6,29,7,64,1,66,False,False
7,191,8,126,1,125,False,True
8,147,9,62,1,59,False,False
9,362,10,62,1,63,False,False


In [15]:
pd.json_normalize(team_data_json, 'chips')

,status_for_entry,played_by_entry,name,number,start_event,stop_event,chip_type
0,played,[25],wildcard,1,21,38,transfer
1,played,[9],freehit,1,2,38,transfer
2,available,[],bboost,1,1,38,team
3,available,[],3xc,1,1,38,team


In [16]:
test = pd.json_normalize(team_data_json, 'chips')
test

,status_for_entry,played_by_entry,name,number,start_event,stop_event,chip_type
0,played,[25],wildcard,1,21,38,transfer
1,played,[9],freehit,1,2,38,transfer
2,available,[],bboost,1,1,38,team
3,available,[],3xc,1,1,38,team


In [17]:
list(test[test['status_for_entry'] == 'available']['name'])

['bboost', '3xc']

In [18]:
team_data_json['transfers']

{'cost': 4, 'status': 'cost', 'limit': 2, 'made': 0, 'bank': 16, 'value': 1001}

In [19]:
player_data_json = _get_fpl_json("https://fantasy.premierleague.com/api/bootstrap-static/")
players_raw = pd.json_normalize(player_data_json, 'elements')

In [20]:
players_raw.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type
0,100.0,100.0,69140,0,0,-4,4,0,2,0.0,...,107.0,29.8,289,113,383,141,246,68,330,117
1,100.0,100.0,98745,0,0,0,0,0,2,0.0,...,38.0,19.5,328,129,329,112,338,118,366,132
2,75.0,75.0,111457,0,0,-3,3,0,2,0.0,...,67.0,41.2,279,110,195,47,294,93,289,101
3,100.0,100.0,154043,0,0,-5,5,1,2,0.0,...,37.0,44.9,225,89,199,48,339,119,271,89
4,100.0,100.0,39476,0,0,-2,2,1,2,0.0,...,110.0,58.5,105,37,331,113,241,65,214,62


In [21]:
player_data_response = session.get("https://fantasy.premierleague.com/api/bootstrap-static/")
player_data_json = json.loads(player_data_response.text)

In [22]:
players_raw = pd.json_normalize(player_data_json, 'elements')
players_raw.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type
0,100.0,100.0,69140,0,0,-4,4,0,2,0.0,...,107.0,29.8,289,113,383,141,246,68,330,117
1,100.0,100.0,98745,0,0,0,0,0,2,0.0,...,38.0,19.5,328,129,329,112,338,118,366,132
2,75.0,75.0,111457,0,0,-3,3,0,2,0.0,...,67.0,41.2,279,110,195,47,294,93,289,101
3,100.0,100.0,154043,0,0,-5,5,1,2,0.0,...,37.0,44.9,225,89,199,48,339,119,271,89
4,100.0,100.0,39476,0,0,-2,2,1,2,0.0,...,110.0,58.5,105,37,331,113,241,65,214,62


In [23]:
players_raw.shape

(628, 61)

In [24]:
players_raw[players_raw['id'] == 215]['second_name']

319    De Bruyne
Name: second_name, dtype: object

In [25]:
team_data = pd.json_normalize(team_data_json, 'picks')
print(team_data.shape)
team_data.head()

(15, 7)


,element,position,selling_price,multiplier,purchase_price,is_captain,is_vice_captain
0,93,1,47,1,46,False,False
1,122,2,53,1,53,False,False
2,182,3,75,1,72,False,False
3,183,4,65,1,65,False,False
4,443,5,54,1,54,False,False


In [26]:
team_data = team_data.merge(
    players_raw[['id', 'first_name', 'second_name']],
    left_on='element',
    right_on='id'
)

team_data['name'] = team_data['first_name'] + '_' + team_data['second_name']
team_data['name'] = team_data['name'].str.lower()

In [27]:
team_data

,element,position,selling_price,multiplier,purchase_price,is_captain,is_vice_captain,id,first_name,second_name,name
0,93,1,47,1,46,False,False,93,Nick,Pope,nick_pope
1,122,2,53,1,53,False,False,122,Aaron,Wan-Bissaka,aaron_wan-bissaka
2,182,3,75,1,72,False,False,182,Trent,Alexander-Arnold,trent_alexander-arnold
3,183,4,65,1,65,False,False,183,Virgil,van Dijk,virgil_van dijk
4,443,5,54,1,54,False,False,443,Rodrigo,Hernandez,rodrigo_hernandez
5,215,6,103,2,101,True,False,215,Kevin,De Bruyne,kevin_de bruyne
6,29,7,64,1,66,False,False,29,Jack,Grealish,jack_grealish
7,191,8,126,1,125,False,True,191,Mohamed,Salah,mohamed_salah
8,147,9,62,1,59,False,False,147,Dominic,Calvert-Lewin,dominic_calvert-lewin
9,362,10,62,1,63,False,False,362,Troy,Deeney,troy_deeney


In [28]:
np.round((team_data['selling_price'].sum() / 10) + (team_data_json['transfers']['bank'] / 10), 2)

99.9